In [1]:
import pandas as pd
import sys
sys.path.append('/home/kehan-vm/Projects/interesting/power_market_analysis/power_market_analysis/src')
from algorithms.battery_optimize import battery_optimisation

def optimize_battery_operation_every_week(
    da_price_data,
    max_battery_capacity,
    initial_capacity,
    end_capacity,
    max_battery_power,
    efficiency,
    daily_max_charging_circles,
    include_revenue,
    solver='glpk'
):
    """
    Optimise the battery operation for every week in the given data
    """
    # Convert datetime to datetime64[ns]
    da_price_data['datetime'] = da_price_data['datetime'].astype('datetime64[ns]')
    
    # Group by week
    da_price_data['year'] = da_price_data['datetime'].dt.year
    da_price_data['week'] = da_price_data['datetime'].dt.isocalendar().week

    # Optimise for each week for every year
    for year in da_price_data['year'].unique():
        year_data = da_price_data[da_price_data['year'] == year]
        total_yearly_result_df = pd.DataFrame()
        for week in year_data['week'].unique():
            week_data = year_data[year_data['week'] == week]
            result = battery_optimisation(
                week_data['datetime'], 
                week_data['da_price'], 
                max_battery_capacity=max_battery_capacity,
                initial_capacity=initial_capacity,
                end_capacity=end_capacity,
                max_battery_power=max_battery_power,
                efficiency=efficiency,
                daily_max_charging_circles=daily_max_charging_circles,
                include_revenue=include_revenue,    
                solver=solver
            )
            result.to_csv(f'/home/kehan-vm/Projects/interesting/power_market_analysis/power_market_analysis/data/optimization/weekly_optimization/battery_operation_{year}_{week}.csv', index=False)
            revenue = result['revenue'].sum()
            print(f'Year: {year}, Week: {week}, Revenue: {revenue}')
            total_yearly_result_df = pd.concat([total_yearly_result_df, result])
        total_yearly_result_df.to_csv(f'/home/kehan-vm/Projects/interesting/power_market_analysis/power_market_analysis/data/optimization/yearly_optimization/battery_operation_{year}.csv', index=False)

In [16]:
da_price_2023 = pd.read_csv('/home/kehan-vm/Projects/interesting/power_market_analysis/power_market_analysis/data/processed/da_price_2023.csv')
da_price_2023.datetime = pd.to_datetime(da_price_2023.datetime)

In [17]:
da_price_2023.head()

,datetime,da_price
0,2023-01-01 00:00:00,75.01
1,2023-01-01 00:15:00,-29.95
2,2023-01-01 00:30:00,-29.98
3,2023-01-01 00:45:00,-30.00
4,2023-01-01 01:00:00,26.81


In [18]:
optimize_battery_operation_every_week(
    da_price_2023,
    max_battery_capacity=2,
    initial_capacity=1, 
    end_capacity=1,
    max_battery_power=1,
    efficiency=0.95,
    daily_max_charging_circles=20000000,
    include_revenue=True,
    solver='glpk')

Year: 2023, Week: 52, Revenue: 3306.4139845394734
Year: 2023, Week: 1, Revenue: 7140.189917763157
Year: 2023, Week: 2, Revenue: 6234.05656381579
Year: 2023, Week: 3, Revenue: 9576.677644736843
Year: 2023, Week: 4, Revenue: 10038.739229605264
Year: 2023, Week: 5, Revenue: 4283.565569407894
Year: 2023, Week: 6, Revenue: 6876.014937828947
Year: 2023, Week: 7, Revenue: 3784.153425657895
Year: 2023, Week: 8, Revenue: 3667.7502506578944
Year: 2023, Week: 9, Revenue: 3662.5635552631575
Year: 2023, Week: 10, Revenue: 2805.6527029605268
Year: 2023, Week: 11, Revenue: 3084.6233845394736
Year: 2023, Week: 12, Revenue: 4672.882708552632
Year: 2023, Week: 13, Revenue: 3299.845323355263
Year: 2023, Week: 14, Revenue: 2945.606199342105
Year: 2023, Week: 15, Revenue: 2656.0467805921053
Year: 2023, Week: 16, Revenue: 3334.2985424342105
Year: 2023, Week: 17, Revenue: 2933.1158184210526
Year: 2023, Week: 18, Revenue: 3083.9555664473683
Year: 2023, Week: 19, Revenue: 2512.817096710526
Year: 2023, Week: 20

In [13]:
df_2024_revenue = pd.read_csv('/home/kehan-vm/Projects/interesting/power_market_analysis/power_market_analysis/data/optimization/yearly_optimization/battery_operation_2024.csv')
df_2024_revenue.head()
df_2024_revenue.revenue.sum()

np.float64(150279.6487819079)

In [14]:
df_2024_first_half = df_2024_revenue[df_2024_revenue.datetime < '2024-07-01']

In [15]:
df_2024_first_half.revenue.sum()

np.float64(66896.1946493421)

In [19]:
df_2024_revenue.head()

,datetime,spot_price,power,market_dispatch,opening_capacity,revenue
0,2024-01-01 00:00:00,39.91,1.0,0.2375,1.0000,9.478625
1,2024-01-01 00:15:00,-0.04,0.0,0.0000,0.7500,-0.000000
2,2024-01-01 00:30:00,-9.01,0.0,0.0000,0.7500,-0.000000
3,2024-01-01 00:45:00,-29.91,-1.0,-0.2500,0.7500,7.477500
4,2024-01-01 01:00:00,25.28,1.0,0.2375,0.9875,6.004000
